# Характер взаимодействия посетителей Яндекс.Дзен с системой

Цель проекта - автоматизировать анализ взаимодействия пользователей с карточками Яндекс.Дзен.
Предполагается, что дашбордом будут пользоваться не реже одного раза в неделю.
Основными пользователями дашборда будут менеджеры по анализу контента.

Состав данных для дашборда:
История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
Разбивка событий по темам источников;
Таблица соответствия тем источников темам карточек.

Данные должны группироваться по:
Дате и времени;
Теме карточки;
Теме источника;
Возрастной группе.

Характер данных в дашборде:
История событий по темам карточек — абсолютные величины с разбивкой по минутам;
Разбивка событий по темам источников — относительные величины (% событий);
Соответствия тем источников темам карточек - абсолютные величины.

Все графики имеют равную важность.

Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw).

База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen.

Частота обновления данных: один раз в сутки, в полночь по UTC.

Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде - указано в макете дашборда.

Структура проекта:
1. [Изучение данных.](#start)
2. [Предобработка данных.](#preprocessing)
3. [Анализ данных.](#analysis)
4. [Общий вывод.](#outcome)


In [62]:
# импортируем библиотеки
import pandas as pd
#! pip install psycopg2
from sqlalchemy import create_engine


In [63]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)



In [64]:
query = '''
        SELECT * FROM dash_visits
        '''

pd.io.sql.read_sql(query, con = engine) 

dash_visits  = pd.io.sql.read_sql(query, con = engine)
 
dash_visits.to_csv('dash_visits.csv')

## Изучение данных.<a id="start"></a>

In [65]:
display(dash_visits.head(10))
dash_visits.info()


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


## Предобработка данных.<a id="preprocessing"></a>

In [66]:
display(dash_visits.isna().sum())
display(dash_visits.duplicated().sum())
display(dash_visits['record_id'].duplicated().sum())

try:
    dash_visits['visits'] = dash_visits['visits'].abs() #Преобразую отрицательные 
#значения в столбце в положительные.
except:
    dash_visits['visits'] = None #В случае ошибки заменяю отрицательные значения значением None.

record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64

0

0

Проверил таблицу на пропуски и дубликаты строк. Пропусков и дубликатов нет. Все значения record_id уникальные.
Убрал отрицательные значения в колонке visits.

## Анализ данных.<a id="analysis"></a>

In [67]:
#Группирую данные таблицы по теме карточек, чтобы получить количество взаимодействий по каждой теме.
dash_visits_group = dash_visits.copy(deep=True)
dash_visits_group = dash_visits_group.groupby('item_topic')['visits'].sum()
display(dash_visits_group)
display(dash_visits_group.median())

item_topic
Деньги                10291
Дети                  10989
Женская психология     7737
Женщины               11499
Здоровье              10399
Знаменитости           7394
Интересные факты      19942
Искусство              8516
История               15389
Красота                9814
Культура              10205
Наука                 21736
Общество              19640
Отношения             20666
Подборки              17772
Полезные советы       15435
Психология             8036
Путешествия            9260
Рассказы              10909
Россия                16966
Семья                 11897
Скандалы               9294
Туризм                 9512
Шоу                    7511
Юмор                   9398
Name: visits, dtype: int64

10399.0

Разброс количества взаимодействий довольно большой от 7511 (Шоу) до 21736 (Наука). Медианное значение - 10399 взаимодействий с карточками по каждой теме.

In [68]:
#Группирую данные таблицы по теме карточек, чтобы получить количество карточек, сгенерированных каждым источником.
dash_visits_group2 = dash_visits.copy(deep=True)
dash_visits_group2 = dash_visits_group2.groupby('source_topic')['item_topic'].count()
display(dash_visits_group2)
display(dash_visits_group2.median())

source_topic
Авто                  1077
Деньги                 973
Дети                  1459
Еда                    912
Здоровье              1243
Знаменитости          1650
Интерьеры              809
Искусство             1228
История               1437
Кино                  1505
Музыка                 750
Одежда                1379
Полезные советы       1578
Политика              1024
Психология            1055
Путешествия           1563
Ремонт                 985
Россия                1687
Сад и дача            1036
Сделай сам             995
Семейные отношения    1822
Семья                 1405
Спорт                 1007
Строительство          758
Технологии             741
Финансы                667
Name: item_topic, dtype: int64

1066.0

Источники с разными темами генерируют от 667 (Финансы) до 1822 карточек (Семейные отношения). Медианное значение этого показателя - 1066.

In [69]:
#Группирую данные таблицы по теме карточек, чтобы получить количество карточек, сгенерированных каждым источником.
dash_visits_group3 = dash_visits.copy(deep=True)
dash_visits_group3 = dash_visits_group3.groupby(['source_topic', 'item_topic'])['visits'].count()
display(dash_visits_group3)

source_topic  item_topic        
Авто          Деньги                52
              Дети                  43
              Женская психология    15
              Женщины               45
              Здоровье              45
                                    ..
Финансы       Семья                 17
              Скандалы               9
              Туризм                30
              Шоу                   51
              Юмор                  25
Name: visits, Length: 644, dtype: int64

## Общий вывод. <a id="outcome"></a>

1. Всего за изучаемый период произошло 310207 взаимодействий. Разбивка по темам с учётом времени визита отражена на графике "История взаимодействий посетителей с карточками по темам (абсолютные значения)". Расчёты в Jupiter Notebook позволяют дополнить эту информацию. Интенсивность взаимодействия зависит от темы карточек. Минимальное значение за изучаемый период у темы "Шоу" - 7511. Максимальное - у темы карточек "Наука" 21736 визитов. Медианное значение показателя - 10399.
2. Эти данные, однако, не могут быть признаны репрезентативными, поскольку изучен период всего в 32 минуты - с 18-28 до 19-00 24 сентября 2019 г. То есть, здесь отражён небольшой отрезок вечернего времени одного буднего дня. Делать серьёзные выводы оснований нет.
3. График "История взаимодействий посетителей с карточками по темам (относительные значения)" даёт информацию по этому же вопросу, отражая долю каждой темы на определённый момент времени в процентах. Здесь видно, что доля темы "Шоу" колеблется от 1.02% до 2.86%, а доля взаимодействий с карточками по теме "Наука" - от 3.57% до 8.18%.
4. Источники с разными темами генерируют за период исследования от 667 карточек ("Финансы") до 1822 ("Семейные отношения"). Медианное значение показателя - 1066. (Расчёты получены в Jupiter Notebook, поскольку, согласно утверждённому в ТЗ макету, Pie chart "График разбивки событий по темам источников" отражает разбивку событий по темам источников). "График разбивки событий по темам источников" показывает разброс долей тем источников от общей массы событий от 10.74% ("Семейные отношения", до 0.85% ("Финансы"))
5. Соотношение тем источников и тем карточек, представленное в "Таблице соответствия тем карточек темам источников", имеет три варианта. Наиболее выраженный - взаимодействие смежных или частично пересекающихся по смыслу тем источников и карточек (Россия - Общество, Путешествия - Рассказы, Полезные советы - Отношения и т.д.). Полное совпадение темы источника с темой карточки может давать меньшее количество визитов (Дети - Дети), чем смежные и даже, казалось бы, не связанные темы (Дети - История, Дети - Психология, Дети - Деньги). Меньше всего визитов даёт сочетание не связанных между собой тем источников с темами карточек (Финансы - Искусство, Сделай сам - Знаменитости и т.п.). Исходя из перечисленных особенностей, можно предположить, что читателей чаще всего инетересуют тексты на стыке тем, комбинирующие разные аспекты жизни.

Dashboard link: https://public.tableau.com/views/Dashboard__16556373096090/sheet0?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

Presentation link: https://drive.google.com/file/d/1lWTjoDL7y5l2mppVGnAxvIQ6JKF48L3D/view?usp=sharing